<a href="https://colab.research.google.com/github/israinamdar493-ops/retinalprojectsizz4/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Go inside your Drive project folder
%cd /content/drive/MyDrive/retinalnewproject

# (Optional) check the structure
!ls -R | head -40   # shows first 40 lines of folder tree

# Install all dependencies
!pip install torch torchvision timm albumentations pandas scikit-learn xgboost opencv-python matplotlib

# >>> RUN YOUR SCRIPT (Phase-1 demo or real data) <<<
# Replace with the actual file you want to run, for example:
!python src/phase1_demo.py


Mounted at /content/drive
/content/drive/MyDrive/retinalnewproject
.:
datasets
ensemble
federated
outputs
preprocessing
reports
results
segmentation

./datasets:
APTOS
CHASEDB1
DRIVE
Messidor2
ODIR5K
RFMiD

./datasets/APTOS:
test.csv
test_images
train_1.csv
train_images
valid.csv
val_images

./datasets/APTOS/test_images:
test_images

./datasets/APTOS/test_images/test_images:
e4dcca36ceb4.png
e50b0174690d.png
e5197d77ec68.png
e529c5757d64.png
e594c19e2e1d.png
e5de79795c1d.png
e60e4edb3ca9.png
e6552b7432b3.png
e66855a5c583.png
e68746d426b2.png
python3: can't open file '/content/drive/MyDrive/retinalnewproject/src/phase1_demo.py': [Errno 2] No such file or directory


In [ ]:
# Save this cell as a .py file inside your Drive
demo_code = """
import torch, torchvision
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

print('✅ Demo running with Torch version:', torch.__version__)

# Define dataset paths - These should match the variables defined in the notebook
train_csv = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv"
train_dir = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images"
val_csv = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/valid.csv"
val_dir = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images"


class RetinalDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.png')
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define a simple transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
try:
    train_dataset = RetinalDataset(train_csv, train_dir, transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    print(f"✅ Loaded training dataset with {len(train_dataset)} images.")

    val_dataset = RetinalDataset(val_csv, val_dir, transform)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    print(f"✅ Loaded validation dataset with {len(val_dataset)} images.")

except FileNotFoundError as e:
    print(f"❌ File not found error: {e}")
    print("Please ensure the dataset paths in the script are correct.")
    exit() # Exit the script if dataset loading fails

# Fake training loop
for epoch in range(1,4):
    print(f'Epoch {epoch}: training...done!')
print('✅ Finished demo training.')
"""
with open('/content/drive/MyDrive/retinalnewproject/phase1_demo.py', 'w') as f:
    f.write(demo_code)

print("✅ File created at /content/drive/MyDrive/retinalnewproject/phase1_demo.py")

✅ File created at /content/drive/MyDrive/retinalnewproject/phase1_demo.py


In [ ]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


✅ Demo running with Torch version: 2.8.0+cu126
Epoch 1: training...done!
Epoch 2: training...done!
Epoch 3: training...done!
✅ Finished demo training.


In [ ]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 58, in <module>
    train_dataset = RetinalDataset(train_csv, train_dir, transform)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 18, in __init__
    self.df = pd.read_csv(csv_file)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^

In [ ]:
import os

base_path = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS"

train_csv = os.path.join(base_path, "train_1.csv")
train_dir = os.path.join(base_path, "train_images")

val_csv = os.path.join(base_path, "valid.csv")
val_dir = os.path.join(base_path, "val_images")

In [ ]:
base_path = "/content/drive/MyDrive/retinalnewproject/datasets/ODIR5K"


In [ ]:
base_path = "/content/drive/MyDrive/retinalnewproject/datasets/RFMiD"


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv")
print(df.columns)


Index(['id_code', 'diagnosis'], dtype='object')


In [ ]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 89, in <module>
    for images, labels in train_loader:
                          ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 790, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 47, in __getitem__
    image = Image.open(img_path).convert('RGB')
            ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages

In [ ]:
# Original
train_dir = os.path.join(base_path, "train_images")
val_dir   = os.path.join(base_path, "val_images")

# Corrected (include the extra 'test_images' folder)
train_dir = os.path.join(base_path, "train_images", "test_images")
val_dir   = os.path.join(base_path, "val_images", "val_images")  # check val_images folder similarly


In [ ]:
!ls /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/


train_images


In [ ]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images
Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 118, in <module>
    for images, labels in train_loader:
                          ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 790, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] 

In [ ]:
# phase1_demo.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# -------------------------------
# 0. Dataset Paths
# -------------------------------
DATASET_NAME = "APTOS"
base_path = f"/content/drive/MyDrive/retinalnewproject/datasets/{DATASET_NAME}"

train_csv = os.path.join(base_path, "train_1.csv")
val_csv   = os.path.join(base_path, "valid.csv")
test_csv  = os.path.join(base_path, "test.csv")  # optional

# -------------------------------
# 1. Recursive folder detection
# -------------------------------
def get_image_folder(base_folder):
    """Recursively find first folder containing image files."""
    for root, dirs, files in os.walk(base_folder):
        img_files = [f for f in files if f.lower().endswith(('.png','.jpg','.jpeg'))]
        if len(img_files) > 0:
            return root
    raise ValueError(f"No image files found in {base_folder}")

train_dir = get_image_folder(os.path.join(base_path, "train_images"))
val_dir   = get_image_folder(os.path.join(base_path, "val_images"))
test_dir  = get_image_folder(os.path.join(base_path, "test_images"))

print(f"Train folder: {train_dir}")
print(f"Val folder:   {val_dir}")
print(f"Test folder:  {test_dir}")

# -------------------------------
# 2. Dataset Class
# -------------------------------
class RetinalDataset(Dataset):
    def __init__(self, csv_file, img_dir, label_col='diagnosis', transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_col = label_col

        self.df['id_code'] = self.df['id_code'].astype(str).str.strip()

        # Map labels to integers if not numeric
        if not np.issubdtype(self.df[self.label_col].dtype, np.integer):
            self.df[self.label_col] = pd.Categorical(self.df[self.label_col]).codes

        # Build mapping from id_code -> full image path
        self.id_to_path = {}
        for root, dirs, files in os.walk(self.img_dir):
            for f in files:
                name, ext = os.path.splitext(f)
                if name in self.df['id_code'].values and ext.lower() in ['.png','.jpg','.jpeg']:
                    self.id_to_path[name] = os.path.join(root, f)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        id_code = row['id_code']
        if id_code not in self.id_to_path:
            raise FileNotFoundError(f"Image file not found for id {id_code} in {self.img_dir}")
        img_path = self.id_to_path[id_code]

        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(row[self.label_col], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# -------------------------------
# 3. Transformations
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# -------------------------------
# 4. Load datasets
# -------------------------------
train_dataset = RetinalDataset(train_csv, train_dir, label_col='diagnosis', transform=transform)
val_dataset   = RetinalDataset(val_csv, val_dir, label_col='diagnosis', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)

# -------------------------------
# 5. Model
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(pd.unique(train_dataset.df['diagnosis']))
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# -------------------------------
# 6. Training loop
# -------------------------------
epochs = 3  # increase for better accuracy
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_loss/len(train_loader):.4f}")

# -------------------------------
# 7. Validation
# -------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1  = f1_score(all_labels, all_preds, average='weighted')
print(f"Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# -------------------------------
# 8. Save model
# -------------------------------
results_path = "/content/drive/MyDrive/retinalnewproject/results"
os.makedirs(results_path, exist_ok=True)
torch.save(model.state_dict(), os.path.join(results_path, f"{DATASET_NAME}_efficientnet_phase1.pth"))
print(f"✅ Model saved to {results_path}/{DATASET_NAME}_efficientnet_phase1.pth")


Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images


FileNotFoundError: Image file not found for id 916ec976ff30 in /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Mounted at /content/drive
Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images
Using 967 images from /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Using 112 images from /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Epoch 1/3 | Train Loss: 1.3486
Epoch 2/3 | Train Loss: 0.7365
Epoch 3/3 | Train Loss: 0.6886
Validation Accuracy: 0.9286 | F1 Score: 0.9157
✅ Model saved to /content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth


In [ ]:
# Install Gradio if not installed
!pip install gradio --quiet

import torch
from torchvision import models, transforms
from PIL import Image
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np

# -------------------
# Settings
# -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 5  # change to your number of disease classes
model_path = "/content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth"

# -------------------
# Image preprocessing
# -------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# -------------------
# Load model
# -------------------
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
model.to(device)

# -------------------
# Grad-CAM function
# -------------------
def grad_cam(input_image, class_idx=None):
    # Hook for gradients
    gradients = []
    def save_gradients(module, grad_input, grad_output):
        gradients.append(grad_output[0])
    target_layer = model.features[-1]
    target_layer.register_backward_hook(save_gradients)

    # Prepare image
    input_tensor = transform(input_image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True

    # Forward pass
    output = model(input_tensor)
    if class_idx is None:
        class_idx = torch.argmax(output, dim=1).item()

    # Backward pass
    model.zero_grad()
    loss = output[0, class_idx]
    loss.backward()

    grads = gradients[0].cpu().data.numpy()[0]
    fmap = target_layer(input_tensor).cpu().data.numpy()[0]

    # Grad-CAM calculation
    weights = np.mean(grads, axis=(1,2))
    cam = np.zeros(fmap.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    cam = np.uint8(cam * 255)
    cam = Image.fromarray(cam).resize(input_image.size, Image.ANTIALIAS)
    return cam

# -------------------
# Prediction + Grad-CAM
# -------------------
def predict_with_gradcam(img):
    # Predict
    input_tensor = transform(img).unsqueeze(0).to(device)
    output = model(input_tensor)
    pred_idx = torch.argmax(output, dim=1).item()

    # Map class index to label (adjust this list to your dataset)
    labels = ['Normal', 'Diabetic Retinopathy', 'Glaucoma', 'Cataract', 'Other']
    pred_label = labels[pred_idx]

    # Grad-CAM overlay
    cam = grad_cam(img, class_idx=pred_idx)
    img_with_cam = Image.blend(img.convert("RGB"), cam.convert("RGB"), alpha=0.5)

    return pred_label, img_with_cam

# -------------------
# Launch Gradio app
# -------------------
interface = gr.Interface(
    fn=predict_with_gradcam,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Textbox(label="Predicted Disease"), gr.Image(label="Grad-CAM Heatmap")],
    title="Retinal Disease Detection",
    description="Upload a fundus image to get predicted disease and Grad-CAM heatmap."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6c32cdd5511314805.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install --upgrade gradio --quiet

import torch
from torchvision import models, transforms
from PIL import Image
import gradio as gr
import numpy as np
import os

# -------------------
# Settings
# -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 5
model_path = "/content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth"

# -------------------
# Preprocessing
# -------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# -------------------
# Load model
# -------------------
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
model.to(device)

# -------------------
# Grad-CAM function
# -------------------
def grad_cam(input_image, class_idx=None):
    gradients = []

    def save_gradients(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    target_layer = model.features[-1]
    target_layer.register_full_backward_hook(save_gradients)

    input_tensor = transform(input_image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True
    output = model(input_tensor)
    if class_idx is None:
        class_idx = torch.argmax(output, dim=1).item()
    model.zero_grad()
    loss = output[0, class_idx]
    loss.backward()

    grads = gradients[0].cpu().data.numpy()[0]
    fmap = target_layer(input_tensor).cpu().data.numpy()[0]
    weights = np.mean(grads, axis=(1, 2))
    cam = np.zeros(fmap.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)
    cam = np.uint8(cam * 255)
    cam = Image.fromarray(cam).resize(input_image.size, Image.ANTIALIAS).convert("RGB")
    return cam

# -------------------
# Prediction + Grad-CAM
# -------------------
labels = ['Normal', 'Diabetic Retinopathy', 'Glaucoma', 'Cataract', 'Other']

def predict_multiple(file_paths):
    preds = []
    cams = []
    if not isinstance(file_paths, list):
        file_paths = [file_paths]

    for f in file_paths:
        img = Image.open(f).convert("RGB")
        input_tensor = transform(img).unsqueeze(0).to(device)
        output = model(input_tensor)
        pred_idx = torch.argmax(output, dim=1).item()
        pred_label = labels[pred_idx]
        preds.append(f"{os.path.basename(f)}: {pred_label}")

        cam = grad_cam(img, class_idx=pred_idx)
        blended = Image.blend(img, cam, alpha=0.5)
        cams.append(blended)

    # Return predictions as a single string
    return "\n".join(preds), cams

# -------------------
# Gradio Interface
# -------------------
interface = gr.Interface(
    fn=predict_multiple,
    inputs=gr.File(file_types=[".png", ".jpg", ".jpeg"], file_count="multiple", label="Upload Fundus Images"),
    outputs=[gr.Textbox(label="Predicted Diseases"), gr.Gallery(label="Grad-CAM Heatmaps")],
    title="Retinal Disease Detection",
    description="Upload one or more fundus images to get predictions and Grad-CAM heatmaps."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5799619c6f7b159e42.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os

# Base dataset folder
dataset_base = "/content/drive/MyDrive/retinalnewproject/datasets"

# List all datasets
for dataset_name in os.listdir(dataset_base):
    dataset_path = os.path.join(dataset_base, dataset_name)
    if os.path.isdir(dataset_path):
        print(f"\nDataset: {dataset_name}")

        # List CSV files
        csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]
        print("  CSV files:")
        for f in csv_files:
            print(f"    {os.path.join(dataset_path, f)}")

        # List image folders and first few images
        img_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
        print("  Image folders:")
        for f in img_folders:
            img_folder_path = os.path.join(dataset_path, f)
            imgs = os.listdir(img_folder_path)
            print(f"    {img_folder_path} ({len(imgs)} files)")
            print(f"      Sample images: {imgs[:5]}")



Dataset: DRIVE
  CSV files:
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/DRIVE/DRIVE (2 files)
      Sample images: ['training', 'test']

Dataset: .ipynb_checkpoints
  CSV files:
  Image folders:

Dataset: APTOS
  CSV files:
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test.csv
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/valid.csv
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images (1 files)
      Sample images: ['val_images']
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images (1 files)
      Sample images: ['train_images']
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images (1 files)
      Sample images: ['test_images']

Dataset: CHASEDB1
  CSV files:
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/CHASEDB1/Images (28 files)
      Sample images: ['Image_12L.

In [ ]:
def get_actual_image_folder(base_folder):
    subfolders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]
    if len(subfolders) == 0:
        raise ValueError(f"No subfolders found in {base_folder}")
    return os.path.join(base_folder, subfolders[0])

train_dir = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images")
val_dir   = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images")
test_dir  = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images")


In [ ]:
# ===============================
# Messidor2 + ResNet50 Training
# ===============================
import os, glob
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, f1_score

# -------- Paths --------
base_path   = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2"
csv_file    = os.path.join(base_path, "messidor_data.csv")   # <-- ensure columns: id_code, diagnosis
img_dir     = os.path.join(base_path, "messidor-2")
results_dir = "/content/drive/MyDrive/retinalnewproject/results"
os.makedirs(results_dir, exist_ok=True)

print("CSV:", csv_file)
print("Images:", img_dir)

# -------- Dataset --------
class MessidorDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        # adapt these if different
        self.img_ids = self.df["id_code"].tolist()
        self.labels  = self.df["diagnosis"].tolist()
        self.img_dir = img_dir
        self.transform = transform

        # map available files for robust loading
        all_imgs = {os.path.splitext(os.path.basename(p))[0]: p
                    for p in glob.glob(os.path.join(img_dir, "*"))}

        self.paths = []
        for i in self.img_ids:
            p = all_imgs.get(str(i))
            if p is None:
                raise FileNotFoundError(f"Image for ID {i} not found in {img_dir}")
            self.paths.append(p)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = Image.open(self.paths[idx]).convert("RGB")
        label = int(self.labels[idx])
        if self.transform: image = self.transform(image)
        return image, label

# -------- Transforms --------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# -------- Data Split --------
full_ds = MessidorDataset(csv_file, img_dir, transform)
train_size = int(0.8 * len(full_ds))
val_size   = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(f"Train size: {len(train_ds)}, Val size: {len(val_ds)}")

# -------- Model --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(full_ds.labels))

model = models.resnet50(weights="IMAGENET1K_V1")  # pretrained
for param in model.parameters():
    param.requires_grad = False  # freeze backbone
model.fc = nn.Linear(model.fc.in_features, num_classes)  # new head
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

# -------- Training --------
epochs = 3
for epoch in range(1, epochs+1):
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*imgs.size(0)
    print(f"Epoch {epoch}/{epochs} | Train Loss: {running_loss/len(train_loader.dataset):.4f}")

# -------- Validation --------
model.eval()
preds, gts = [], []
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds.extend(outputs.argmax(dim=1).cpu().numpy())
        gts.extend(labels.numpy())

acc = accuracy_score(gts, preds)
f1  = f1_score(gts, preds, average="weighted")
print(f"Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# -------- Save --------
save_path = os.path.join(results_dir, "Messidor2_resnet50_phase1.pth")
torch.save(model.state_dict(), save_path)
print(f"✅ Model saved to {save_path}")


CSV: /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv
Images: /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2


FileNotFoundError: Image for ID 20051020_43808_0100_PP.png not found in /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2

In [ ]:
import os, glob

img_dir = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2"
files = [os.path.basename(p) for p in glob.glob(os.path.join(img_dir, "*"))]

print("Total images found:", len(files))
print("Sample names:", files[:10])


Total images found: 1
Sample names: ['messidor-2']


In [ ]:
import os

root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2"
print("Contents:", os.listdir(root))


Contents: ['messidor-2']


In [ ]:
import os
deep_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2"
print("Contents:", os.listdir(deep_path))


Contents: ['preprocess']


In [ ]:
import os

deepest_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess"
print("Contents:", os.listdir(deepest_path)[:20])  # show first 20 entries


Contents: ['20060412_51775_0200_PP.png', '20060411_61979_0200_PP.png', '20060411_62142_0200_PP.png', '20060411_59549_0200_PP.png', '20060411_59812_0200_PP.png', '20060412_52425_0200_PP.png', '20060412_51746_0200_PP.png', '20060412_52371_0200_PP.png', '20060411_61060_0200_PP.png', '20060411_59747_0200_PP.png', '20060411_61402_0200_PP.png', '20060411_58413_0200_PP.png', '20060411_58494_0200_PP.png', '20060411_58993_0200_PP.png', '20060411_62036_0200_PP.png', '20060411_59176_0200_PP.png', '20060412_52020_0200_PP.png', '20060411_60426_0200_PP.png', '20060411_60028_0200_PP.png', '20060412_52351_0200_PP.png']


In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# -------------------------
# Paths  ✅  (UPDATED)
# -------------------------
csv_file = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
img_dir  = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess"

# -------------------------
# Dataset
# -------------------------
class MessidorDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        # Build a map for quick lookup
        all_imgs = {os.path.basename(x): os.path.join(root, x)
                    for root, _, files in os.walk(img_dir)
                    for x in files}

        self.paths, self.labels = [], []
        for _, row in self.data.iterrows():
            img_name = row['image']
            label = row['label']
            p = all_imgs.get(img_name)
            if p is None:
                raise FileNotFoundError(f"Image for ID {img_name} not found in {img_dir}")
            self.paths.append(p)
            self.labels.append(int(label))

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

# -------------------------
# Transforms
# -------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# -------------------------
# Data Split
# -------------------------
full_ds = MessidorDataset(csv_file, img_dir, transform)
train_size = int(0.8 * len(full_ds))
val_size   = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(f"✅ Total images: {len(full_ds)} | Train: {train_size} | Val: {val_size}")

# -------------------------
# Model
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # Assuming 2 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# -------------------------
# Training Loop
# -------------------------
for epoch in range(3):  # change epochs if needed
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/3] | Train Loss: {running_loss/len(train_loader):.4f}")

# -------------------------
# Validation
# -------------------------
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
f1  = f1_score(all_labels, all_preds, average='macro')
print(f"✅ Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# Save model
os.makedirs("/content/drive/MyDrive/retinalnewproject/results", exist_ok=True)
torch.save(model.state_dict(),
           "/content/drive/MyDrive/retinalnewproject/results/Messidor2_efficientnet_phase1.pth")
print("💾 Model saved successfully!")


KeyError: 'image'

In [ ]:
import pandas as pd

csv_file = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
df = pd.read_csv(csv_file)
print("Columns in CSV:", df.columns.tolist())
print(df.head())


Columns in CSV: ['id_code', 'diagnosis', 'adjudicated_dme', 'adjudicated_gradable']
                      id_code  diagnosis  adjudicated_dme  \
0  20051020_43808_0100_PP.png          0                0   
1  20051020_43832_0100_PP.png          1                0   
2  20051020_43882_0100_PP.png          1                0   
3  20051020_43906_0100_PP.png          2                1   
4  20051020_44261_0100_PP.png          0                0   

   adjudicated_gradable  
0                     1  
1                     1  
2                     1  
3                     1  
4                     1  


In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class MessidorDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        # Build a mapping of all images present in the directory for fast lookup
        all_imgs = {f: os.path.join(root, f)
                    for root, _, files in os.walk(img_dir)
                    for f in files}

        self.paths, self.labels = [], []
        for _, row in self.data.iterrows():
            img_name = row['id_code']       # already includes .png
            label    = row['diagnosis']     # use the diagnosis column
            p = all_imgs.get(img_name)
            if p is None:
                raise FileNotFoundError(f"Image {img_name} not found in {img_dir}")
            self.paths.append(p)
            self.labels.append(label)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.long)


In [ ]:
img_dir = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/preprocess"
csv_file = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"


In [ ]:
# Paste this whole cell into Colab (below the drive.mount cell)
import os, glob, time
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, f1_score

# ---------- SETTINGS ----------
BASE = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2"
CSV = os.path.join(BASE, "messidor_data.csv")
RESULTS_DIR = "/content/drive/MyDrive/retinalnewproject/results"
os.makedirs(RESULTS_DIR, exist_ok=True)
SEED = 42
torch.manual_seed(SEED)

# ---------- Helper: find folder that actually contains images ----------
def find_image_root(base_folder, min_images=10):
    """Walk base_folder recursively and return the first folder that has at least min_images image files."""
    exts = (".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp")
    for root, dirs, files in os.walk(base_folder):
        imgs = [f for f in files if f.lower().endswith(exts)]
        if len(imgs) >= min_images:
            return root
    # fallback: return deepest folder with any images
    best = None
    best_count = 0
    for root, dirs, files in os.walk(base_folder):
        imgs = [f for f in files if f.lower().endswith(exts)]
        if len(imgs) > best_count:
            best_count = len(imgs); best = root
    return best

img_root = find_image_root(BASE, min_images=20)
print("Using image root:", img_root)
if img_root is None:
    raise RuntimeError(f"No image files found under {BASE} — check your dataset location.")

# ---------- Inspect CSV ----------
df = pd.read_csv(CSV)
print("CSV columns:", df.columns.tolist())
print(df.head())

# ---------- Dataset class (robust lookup by basename) ----------
class MessidorDataset(Dataset):
    def __init__(self, csv_path, img_root, img_col='id_code', label_col='diagnosis', transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_root = img_root
        self.img_col = img_col
        self.label_col = label_col
        self.transform = transform

        # Build recursive mapping basename -> full path
        self.all_imgs = {}
        for p in glob.glob(os.path.join(self.img_root, "**", "*.*"), recursive=True):
            name = os.path.basename(p)
            stem = os.path.splitext(name)[0]
            # prefer the exact name key (with extension) AND also store stem key for extension-agnostic lookup
            self.all_imgs[name] = p
            self.all_imgs[stem] = p

        # Prepare final lists, and collect missing ids (if any)
        self.paths = []
        self.labels = []
        missing = []
        for _, row in self.df.iterrows():
            raw_name = str(row[self.img_col]).strip()
            # try exact match first (filename may include extension), then stem match
            if raw_name in self.all_imgs:
                p = self.all_imgs[raw_name]
            else:
                stem = os.path.splitext(raw_name)[0]
                p = self.all_imgs.get(stem)
            if p is None:
                missing.append(raw_name)
            else:
                self.paths.append(p)
                self.labels.append(int(row[self.label_col]))
        if len(missing) > 0:
            print(f"⚠️ Warning: {len(missing)} CSV image ids not found in {img_root}.")
            print(" Sample missing ids:", missing[:10])
        print(f"Dataset: found {len(self.paths)} images (skipped {len(missing)} missing)")
        if len(self.paths) == 0:
            raise RuntimeError("No images matched — check CSV 'id_code' values and image folder.")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return img, label

# ---------- Transforms ----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ---------- Build datasets / loaders ----------
full_ds = MessidorDataset(CSV, img_root, img_col='id_code', label_col='diagnosis', transform=transform)
train_size = int(0.8 * len(full_ds))
val_size = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

# ---------- Sanity check: one batch ----------
try:
    batch_imgs, batch_labels = next(iter(train_loader))
    print("Sanity batch shapes:", batch_imgs.shape, batch_labels.shape)
except Exception as e:
    print("Sanity check failed:", e)

# ---------- Model: ResNet-50 (transfer learning) ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(full_ds.labels))
print("Num classes:", num_classes)

model = models.resnet50(pretrained=True)
# freeze backbone (optional) — comment these two lines to fine-tune everything
for param in model.parameters():
    param.requires_grad = False
# replace the classifier
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

# ---------- Training loop (small run) ----------
EPOCHS = 3
best_val_acc = 0.0
metrics_rows = []
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    model.train()
    total_loss = 0.0
    for imgs, labels in train_loader:
        imgs = imgs.to(device); labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    all_preds, all_gts = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_gts.extend(labels.numpy())
    val_acc = accuracy_score(all_gts, all_preds)
    val_f1  = f1_score(all_gts, all_preds, average='macro')
    metrics_rows.append([epoch, train_loss, val_acc, val_f1])
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_path = os.path.join(RESULTS_DIR, "Messidor2_resnet50_best.pth")
        torch.save(model.state_dict(), best_path)
    print(f"Epoch {epoch}/{EPOCHS} | train_loss={train_loss:.4f} | val_acc={val_acc:.4f} | val_f1={val_f1:.4f} | time={time.time()-t0:.1f}s")

# Save final model & metrics CSV
final_path = os.path.join(RESULTS_DIR, "Messidor2_resnet50_final.pth")
torch.save(model.state_dict(), final_path)
metrics_df = pd.DataFrame(metrics_rows, columns=["epoch","train_loss","val_acc","val_f1"])
metrics_df.to_csv(os.path.join(RESULTS_DIR, "Messidor2_metrics.csv"), index=False)
print("Saved final model:", final_path)
print("Saved metrics:", os.path.join(RESULTS_DIR, "Messidor2_metrics.csv"))


Using image root: /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess
CSV columns: ['id_code', 'diagnosis', 'adjudicated_dme', 'adjudicated_gradable']
                      id_code  diagnosis  adjudicated_dme  \
0  20051020_43808_0100_PP.png          0                0   
1  20051020_43832_0100_PP.png          1                0   
2  20051020_43882_0100_PP.png          1                0   
3  20051020_43906_0100_PP.png          2                1   
4  20051020_44261_0100_PP.png          0                0   

   adjudicated_gradable  
0                     1  
1                     1  
2                     1  
3                     1  
4                     1  
Dataset: found 1744 images (skipped 0 missing)
Train: 1395, Val: 349
Sanity batch shapes: torch.Size([16, 3, 224, 224]) torch.Size([16])
Num classes: 5


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s]


Epoch 1/3 | train_loss=1.1092 | val_acc=0.5415 | val_f1=0.2442 | time=528.9s
Epoch 2/3 | train_loss=1.0351 | val_acc=0.5387 | val_f1=0.2380 | time=509.9s


In [ ]:
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
/content/drive/MyDrive/retinalnewproject/results/
    Messidor2_resnet50_best.pth    ← best model (from val accuracy)
    Messidor2_resnet50_final.pth   ← final model
    Messidor2_metrics.csv          ← training metrics


IndentationError: unexpected indent (ipython-input-3146123459.py, line 2)

In [ ]:
# === Fine-tune Messidor2 with ResNet50 ===
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import models

# Load the same dataset objects you already defined:
# train_loader, val_loader, MessidorDataset etc.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Replace final layer (Messidor2 has 5 classes)
model.fc = nn.Linear(model.fc.in_features, 5)

# ---- UNFREEZE the backbone ----
for param in model.parameters():
    param.requires_grad = True

model = model.to(device)

# Adam optimizer with smaller LR for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

EPOCHS = 10
best_acc = 0

for epoch in range(EPOCHS):
    # Training
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {running_loss/len(train_loader.dataset):.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/retinalnewproject/results/Messidor2_resnet50_finetuned_best.pth")
        print("✅ Saved new best model")

print("Best validation accuracy:", best_acc)


Epoch 1/10 | Train Loss: 0.9973 | Val Acc: 0.6619
✅ Saved new best model
Epoch 2/10 | Train Loss: 0.8040 | Val Acc: 0.6734
✅ Saved new best model


KeyboardInterrupt: 

In [1]:
# === Fine-tune Messidor2 with ResNet50 (Optimized) ===
import torch
from torch import nn, optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 5)   # Messidor2 has 5 classes
model = model.to(device)

# Unfreeze all layers
for param in model.parameters():
    param.requires_grad = True

# Optimizer + Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

EPOCHS = 10
best_acc = 0

for epoch in range(EPOCHS):
    # Training loop
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Prevent OOM / slowdown
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    # Validation loop
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    train_loss = running_loss / len(train_loader.dataset)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/retinalnewproject/results/Messidor2_resnet50_finetuned_best.pth"
        )
        print("✅ Saved new best model")

print("Best validation accuracy:", best_acc)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 91.6MB/s]


NameError: name 'train_loader' is not defined

In [3]:
import os
import pandas as pd
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image


In [4]:
class MessidorDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'])
        image = Image.open(img_path).convert("RGB")
        label = int(row['diagnosis'])

        if self.transform:
            image = self.transform(image)
        return image, label


In [5]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [6]:
from sklearn.model_selection import train_test_split

csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/messidor_labels.csv"
img_root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess"

df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['diagnosis'], random_state=42)

train_csv = "/content/train_split.csv"
val_csv = "/content/val_split.csv"
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/messidor_labels.csv'

In [13]:
from sklearn.model_selection import train_test_split

csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
img_root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/preprocess"

df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['diagnosis'], random_state=42)

train_csv = "/content/train_split.csv"
val_csv = "/content/val_split.csv"
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)


In [14]:
train_dataset = MessidorDataset(train_csv, img_root, transform=transform_train)
val_dataset = MessidorDataset(val_csv, img_root, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


In [15]:
# === Step 6 to 9: Training ResNet50 on Messidor2 ===
import torch
from torch import nn, optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load pretrained ResNet50 ---
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Replace final classification layer (Messidor2 has 5 classes)
model.fc = nn.Linear(model.fc.in_features, 5)
model = model.to(device)

# --- Loss & Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 10
best_acc = 0

for epoch in range(EPOCHS):
    # ---- Training ----
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    # ---- Validation ----
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    train_loss = running_loss / len(train_loader.dataset)

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        save_path = "/content/drive/MyDrive/retinalnewproject/results/Messidor2_resnet50_best.pth"
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved new best model at epoch {epoch+1}")

print("🎯 Best validation accuracy:", best_acc)


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-1624774340.py", line 13, in __getitem__
    image = Image.open(img_path).convert("RGB")
            ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/Image.py", line 3513, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/preprocess/IM001759.JPG'


In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


NameError: name 'train_ds' is not defined

In [17]:
# === Step 4 & 5: Dataset split and loaders for Messidor2 ===
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

# Paths
csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
img_root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/preprocess"

# Load CSV
df = pd.read_csv(csv_path)

# Train/val split
train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df['diagnosis'], random_state=42
)

# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Dataset class
class MessidorDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'])
        image = Image.open(img_path).convert("RGB")
        label = int(row['diagnosis'])
        if self.transform:
            image = self.transform(image)
        return image, label

# Create datasets
train_ds = MessidorDataset(train_df, img_root, transform)
val_ds   = MessidorDataset(val_df, img_root, transform)

# Create loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_ds)}, Val samples: {len(val_ds)}")


Train samples: 1395, Val samples: 349


In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pretrained ResNet50
model = models.resnet50(pretrained=True)

# Change the final layer for 5 Messidor classes
num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model ready on:", device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 146MB/s]


Model ready on: cuda


In [2]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [3]:
num_epochs = 5  # you can increase later

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")


NameError: name 'train_loader' is not defined

In [4]:
# === Step 4 & 5: Dataset split and loaders for Messidor2 ===
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

# Paths (Ensure these paths are correct for your setup)
csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
img_root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/preprocess"

# Load CSV
df = pd.read_csv(csv_path)

# Train/val split
# Note: Stratify requires that each class has at least one sample in both train and validation sets.
# If you have classes with very few samples, you might need to adjust the split size or handle them separately.
try:
    train_df, val_df = train_test_split(
        df, test_size=0.2, stratify=df['diagnosis'], random_state=42
    )
except ValueError as e:
    print(f"Error during train/val split: {e}")
    print("Check if your 'diagnosis' column has at least two samples for each class to enable stratification.")
    # Fallback to non-stratified split if stratification fails
    print("Attempting non-stratified split...")
    train_df, val_df = train_test_split(
        df, test_size=0.2, random_state=42
    )


# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Dataset class
class MessidorDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

        # Build a mapping of all images present in the directory for fast lookup
        # This assumes image filenames in the directory exactly match 'id_code' in the CSV
        self.all_imgs = {f: os.path.join(root, f)
                         for root, _, files in os.walk(img_dir)
                         for f in files}

        self.paths, self.labels = [], []
        missing_images = []
        for _, row in self.df.iterrows():
            img_name = str(row['id_code']).strip() # Ensure it's string and strip whitespace
            label    = row['diagnosis']     # use the diagnosis column

            # Check if the image file exists in the pre-built map
            p = self.all_imgs.get(img_name)

            if p is None:
                missing_images.append(img_name)
                # Optionally skip this row or handle the missing image
                # print(f"Warning: Image {img_name} not found in {img_dir}. Skipping.")
                continue # Skip this image

            self.paths.append(p)
            self.labels.append(int(label)) # Ensure label is integer

        if missing_images:
            print(f"⚠️ Warning: {len(missing_images)} images from the dataframe were not found in the image directory.")
            print("Sample missing images:", missing_images[:10])

        if not self.paths:
             raise RuntimeError("No images loaded. Please check CSV file and image directory.")


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = self.labels[idx] # Label is already an integer

            if self.transform:
                image = self.transform(image)
            return image, torch.tensor(label, dtype=torch.long)

        except FileNotFoundError:
            print(f"Error: Image file not found at {img_path}. This should not happen if all_imgs map is correct.")
            # Handle this error, maybe return a dummy sample or None, depending on how you want to handle it
            # For simplicity, we'll re-raise or return None. Returning None requires handling in DataLoader.
            # A common practice is to filter out missing images during dataset initialization.
            raise # Re-raise the error if it occurs unexpectedly


# Create datasets
train_ds = MessidorDataset(train_df, img_root, transform)
val_ds   = MessidorDataset(val_df, img_root, transform)

# Create loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_ds)}, Val samples: {len(val_ds)}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv'

In [23]:
# === Messidor2 Fine-tuning with ResNet50 ===
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import glob # Import glob for recursive file search

# ---------------- Paths ----------------
csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
# Correcting the image root to the preprocess folder
img_root = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess"

# --------------- Dataset ----------------
class MessidorDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

        # Build a mapping of all images present in the directory recursively
        self.all_imgs = {}
        # Searching for common image extensions recursively
        print(f"Searching for images in: {self.img_dir}")
        found_files_count = 0
        for ext in ['*.png', '*.jpg', '*.jpeg', '*.JPG']:
             for img_path in glob.glob(os.path.join(self.img_dir, '**', ext), recursive=True):
                img_name = os.path.basename(img_path)
                self.all_imgs[img_name] = img_path
                # Also add the filename without extension for more flexible lookup
                stem, _ = os.path.splitext(img_name)
                self.all_imgs[stem] = img_path
                found_files_count += 1

        print(f"Found {found_files_count} image files in total.")
        # Print a few found files to verify
        print("Sample files found:", list(self.all_imgs.keys())[:20])


        self.paths, self.labels = [], []
        missing_images = []
        print("Matching CSV entries with found images...")
        for index, row in self.df.iterrows():
            # The id_code in Messidor2 CSV includes the extension, e.g., 'IM000001.jpg'
            img_name_in_csv = str(row['id_code']).strip()
            label    = row['diagnosis']

            # Try to match the image file using the name from the CSV (exact match first)
            p = self.all_imgs.get(img_name_in_csv)

            # If exact match fails, try matching without extension
            if p is None:
                stem_in_csv, _ = os.path.splitext(img_name_in_csv)
                p = self.all_imgs.get(stem_in_csv)


            if p is None:
                missing_images.append(img_name_in_csv)
                # print(f"Warning: Image {img_name_in_csv} not found in {self.img_dir}. Skipping.")
                continue # Skip this image

            self.paths.append(p)
            self.labels.append(int(label)) # Ensure label is integer

        if missing_images:
            print(f"⚠️ Warning: {len(missing_images)} images from the dataframe were not found in the image directory.")
            print("Sample missing images:", missing_images[:10])

        if not self.paths:
             raise RuntimeError("No images loaded. Please check CSV file and image directory, and image filenames/extensions.")

        print(f"Successfully matched {len(self.paths)} images from CSV.")


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = self.labels[idx]

            if self.transform:
                image = self.transform(image)
            return image, torch.tensor(label, dtype=torch.long)

        except FileNotFoundError:
            # This should theoretically not happen if all_imgs map is built correctly
            print(f"Error: Image file not found at {img_path} during loading.")
            raise # Re-raise the error


# ------------- Transforms ---------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]) # Added normalization
])

# -------- Load CSV and split -----------
df = pd.read_csv(csv_path)
# Handle potential stratification error if needed (re-added from previous attempt)
try:
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['diagnosis'], random_state=42)
except ValueError as e:
    print(f"Error during stratified train/val split: {e}. Attempting non-stratified split.")
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


# ---------- Dataset and DataLoader ------
train_ds = MessidorDataset(train_df, img_root, transform)
val_ds   = MessidorDataset(val_df, img_root, transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(f"Train samples loaded: {len(train_ds)}, Val samples loaded: {len(val_ds)}")


# --------------- Device ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Model training device:", device)

# --------------- Model -----------------
# Using pretrained weights
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 5)  # 5 classes for Messidor2
model = model.to(device)

# --------- Loss & Optimizer -------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ---------------- Training ----------------
EPOCHS = 10
best_acc = 0

print("Starting training...")
for epoch in range(EPOCHS):
    # Train
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {running_loss/len(train_loader.dataset):.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/retinalnewproject/results/Messidor2_resnet50_best.pth")
        print("✅ Saved new best model")

print("Training complete. Best validation accuracy:", best_acc)

Searching for images in: /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess
Found 1744 image files in total.
Sample files found: ['20060412_51775_0200_PP.png', '20060412_51775_0200_PP', '20060411_61979_0200_PP.png', '20060411_61979_0200_PP', '20060411_62142_0200_PP.png', '20060411_62142_0200_PP', '20060411_59549_0200_PP.png', '20060411_59549_0200_PP', '20060411_59812_0200_PP.png', '20060411_59812_0200_PP', '20060412_52425_0200_PP.png', '20060412_52425_0200_PP', '20060412_51746_0200_PP.png', '20060412_51746_0200_PP', '20060412_52371_0200_PP.png', '20060412_52371_0200_PP', '20060411_61060_0200_PP.png', '20060411_61060_0200_PP', '20060411_59747_0200_PP.png', '20060411_59747_0200_PP']
Matching CSV entries with found images...
Successfully matched 1395 images from CSV.
Searching for images in: /content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess
Found 1744 image files in total.
Sample files found: ['20060412_5

In [17]:
# === Messidor2 Training with EfficientNet-B3 and Augmentations ===
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from collections import Counter

# ---------------- Dataset ----------------
class MessidorDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        # Keep only images that exist in folder
        self.df['exists'] = self.df['id_code'].apply(lambda x: os.path.isfile(os.path.join(img_dir, x)))
        self.df = self.df[self.df['exists']]
        self.df = self.df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'])
        label = row['diagnosis']

        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# ---------------- Paths ----------------
csv_path = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor_data.csv"
img_dir  = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2/messidor-2/messidor-2/preprocess"

# ---------------- Transforms ----------------
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ---------------- Dataset and Dataloader ----------------
full_dataset = MessidorDataset(csv_path, img_dir, transform=train_transform)
train_size = int(0.8*len(full_dataset))
val_size   = len(full_dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Assign validation transform
val_ds.dataset.transform = val_transform

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

# ---------------- Weighted Loss ----------------
labels = [y for _, y in train_ds]
class_counts = Counter(labels)
class_weights = [1.0/count for count in class_counts.values()]
class_weights = torch.FloatTensor(class_weights).to('cuda')

# ---------------- Model ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)


# ---------------- Training ----------------
EPOCHS = 15
best_acc = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {running_loss/len(train_loader.dataset):.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Step LR scheduler
    scheduler.step(val_acc)

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/retinalnewproject/results/Messidor2_efficientnetB3_best.pth")
        print("✅ Saved new best model")

print("Best validation accuracy:", best_acc)


Epoch 1/15 | Train Loss: 1.5327 | Val Acc: 0.5244
✅ Saved new best model
Epoch 2/15 | Train Loss: 1.1710 | Val Acc: 0.6103
✅ Saved new best model
Epoch 3/15 | Train Loss: 0.8751 | Val Acc: 0.6332
✅ Saved new best model
Epoch 4/15 | Train Loss: 0.6750 | Val Acc: 0.6275
Epoch 5/15 | Train Loss: 0.5292 | Val Acc: 0.6791
✅ Saved new best model
Epoch 6/15 | Train Loss: 0.3397 | Val Acc: 0.6304
Epoch 7/15 | Train Loss: 0.2844 | Val Acc: 0.6246
Epoch 8/15 | Train Loss: 0.2076 | Val Acc: 0.6562
Epoch 9/15 | Train Loss: 0.1726 | Val Acc: 0.6132
Epoch 10/15 | Train Loss: 0.1404 | Val Acc: 0.6590
Epoch 11/15 | Train Loss: 0.1286 | Val Acc: 0.6504
Epoch 12/15 | Train Loss: 0.0942 | Val Acc: 0.6562
Epoch 13/15 | Train Loss: 0.0999 | Val Acc: 0.6418


KeyboardInterrupt: 

In [14]:
model.load_state_dict(torch.load(
    "/content/drive/MyDrive/retinalnewproject/results/Messidor2_resnet50_best.pth"))
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os

# List files in the Messidor2 dataset directory
messidor2_dir = "/content/drive/MyDrive/retinalnewproject/datasets/Messidor2"
if os.path.exists(messidor2_dir):
    print(f"Contents of {messidor2_dir}:")
    print(os.listdir(messidor2_dir))
else:
    print(f"Directory not found: {messidor2_dir}")

Contents of /content/drive/MyDrive/retinalnewproject/datasets/Messidor2:
['messidor_data.csv', 'messidor-2']
